In [61]:
import sys
import glob
import os
from Bio.Seq import Seq
from Bio import SeqIO, AlignIO, Align, SeqRecord
from collections import Counter


In [18]:
def noise_filter(column):
    """
    noise_filter is a function that determines whether an alignment column is noisy or not.
    :param column: multiple sequence alignment column.
    :return: boolean.
    """
    indel = '-'
    amino_counter = Counter(column)
    if column.count(indel) > len(column) / 2:
        return True
    if len([key for key, value in amino_counter.items() if value == 1 ]) >= len(column) / 2:
        return True
    if max(amino_counter.values()) <= 2:
        return True
    return False     

In [97]:
def reduce_noise(multiple_seq_alignment, alignment_path):
    """
    reduce_noise is a function that filters all noisy columns.
    :param multiple_seq_alignment: MultipleSeqAlignment.
    :return: a MultipleSeqAlignment.
    """
    noise_free_column_list = []
    new_aligned_list = []
    for i in range(multiple_seq_alignment.get_alignment_length()):
        if not noise_filter(multiple_seq_alignment[:, i]):
            noise_free_column_list.append(multiple_seq_alignment[:, i])
    for i in range(len(multiple_seq_alignment)):
        new_aligned_list.append(''.join([x[i] for x in noise_free_column_list]))
    
    record_list = Align.MultipleSeqAlignment([
        SeqRecord.SeqRecord(Seq(new_seq), id=record.id, name=record.name, description=record.description)
        for new_seq, record in zip(new_aligned_list, multiple_seq_alignment)
     ])
    alignment_filename = alignment_path.split('/')[-1]
    if multiple_seq_alignment.get_alignment_length() == record_list.get_alignment_length():
        print(f'{alignment_filename}'+ '>> WARNING: no noise reduction')
    if (record_list.get_alignment_length()  < multiple_seq_alignment.get_alignment_length() / 2 and 
        multiple_seq_alignment.get_alignment_length() - record_list.get_alignment_length() > 0):
        print(f'{alignment_filename}'+ '>> WARNING: more than 0.5 of the sequence is noise')
    return record_list

In [98]:
def perform_noise_reduction(alignment_path):
    with open(alignment_path, mode='r') as aligned_file:
        my_sequence_recorded = AlignIO.read(aligned_file, 'fasta')
    return reduce_noise(my_sequence_recorded, alignment_path)

In [43]:
with open('./test_data/test_more_0.5_noise.msl', mode='r') as aligned_file:
    my_sequence_recorded2 = AlignIO.read(aligned_file, 'fasta')

In [101]:
path = './test_data/test_no_noise.msl'
perform_noise_reduction(path)


test_no_noise.msl>> WARNING: no noise reduction


<<class 'Bio.Align.MultipleSeqAlignment'> instance (16 records of length 358, Alphabet()) at 10ae96fd0>

In [93]:
af = './test_data/test_empty.msl'
perform_noise_reduction(af)

test_empty.msl>> ERROR: empty file.


ValueError: No records found in handle

In [14]:
original_dir = '/Users/user/Desktop/MSM/03-Applied_Bioinformatics/Labs/03_Project/protein_multial_noise_reduction/data/test_data'
reduced_dir = '/Users/user/Desktop/MSM/03-Applied_Bioinformatics/Labs/03_Project/protein_multial_noise_reduction/data/reduced_test_data'
result_dir =  '/Users/user/Desktop/MSM/03-Applied_Bioinformatics/Labs/03_Project/protein_multial_noise_reduction/results'
directories = []

for folder in glob.glob(original_dir +'/*'):
    sub_folder_name = folder.split('/')[-1]
    directories.append(sub_folder_name)
    
for directory in directories:  
    original_dir_path = os.path.join(original_dir, directory)
    new_folder_path = os.path.join(reduced_dir, directory)
    os.makedirs(new_folder_path, exist_ok=True)

    for alignment_path in glob.glob(original_dir_path + '/*.msl'):
        alignment_filename = alignment_path.split('/')[-1]
        if os.stat(alignment_path).st_size == 0:
            sys.exit(f'{alignment_filename}'+ '>> ERROR: empty file.')
        return Align.MultipleSeqAlignment(SeqRecord.SeqRecord(Seq('')))
        
        record_list = perform_noise_reduction(alignment_path)
        if record_list.get_alignment_length() == 0:
            sys.exit(f'{alignment_filename}'+ '>> WARNING: all columns are noisy.')
            
        reduced_filename_out = os.path.join(new_folder_path, alignment_filename)
        AlignIO.write(record_list, reduced_filename_out, "fasta")
    
    
    